Check numberplate detection on dataset from
https://github.com/detectRecog/CCPD

#### Dataset Annotations

Annotations are embedded in file name.

A sample image name is "025-95_113-154&383_386&473-386&473_177&454_154&383_363&402-0_0_22_27_27_33_16-37-15.jpg". Each name can be splited into seven fields. Those fields are explained as follows.

- **Area**: Area ratio of license plate area to the entire picture area.

- **Tilt degree**: Horizontal tilt degree and vertical tilt degree.

- **Bounding box coordinates**: The coordinates of the left-up and the right-bottom vertices.

- **Four vertices locations**: The exact (x, y) coordinates of the four vertices of LP in the whole image. These coordinates start from the right-bottom vertex.

- **License plate number**: Each image in CCPD has only one LP. Each LP number is comprised of a Chinese character, a letter, and five letters or numbers. A valid Chinese license plate consists of seven characters: province (1 character), alphabets (1 character), alphabets+digits (5 characters). "0_0_22_27_27_33_16" is the index of each character. These three arrays are defined as follows. The last character of each array is letter O rather than a digit 0. We use O as a sign of "no character" because there is no O in Chinese license plate characters.
```
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
             'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
       'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']
```

- **Brightness**: The brightness of the license plate region.

- **Blurriness**: The Blurriness of the license plate region.


In [1]:
import os
import sys

NOMEROFF_NET_PATH = "../../"
sys.path.append(NOMEROFF_NET_PATH)

In [3]:
from nomeroff_net.pipes.number_plate_localizators.yolo_v5_detector import Detector

In [ ]:
detector = Detector()
detector.load("/var/www/nomeroff-net/best_china.pt")

In [8]:
import glob
import cv2
import copy
import tqdm
from shutil import copyfile
from matplotlib import pyplot as plt

In [9]:
def parse_file_name_ccpd(
    name="025-95_113-154&383_386&473-386&473_177&454_154&383_363&402-0_0_22_27_27_33_16-37-15.jpg",
    provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", 
              "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"],
    alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
              'X', 'Y', 'Z', 'O'],
    ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
        'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']):

    name = os.path.basename(name)
    name = name.split(".")[0]
    area, tilt_degrees, bbox, vertices, plate_number, brightness, blurriness  = name.split("-")
    
    area = int(area)
    brightness = int(brightness)
    blurriness = int(blurriness)
    
    horizontal_tilt_degree, vertical_tilt_degree = tilt_degrees.split("_")
    horizontal_tilt_degree = int(horizontal_tilt_degree)
    vertical_tilt_degree = int(vertical_tilt_degree)
    
    left_up, right_bottom = bbox.split("_")
    bbox = [*left_up.split("&"), *right_bottom.split("&")]
    bbox = [int(item) for item in bbox]
    
    vertices = vertices.split("_")
    vertices = [[int(coord) for coord in p.split("&")] for p in vertices]
    
    plate_number = "".join([ads[int(idx)] for idx in plate_number.split("_")])
    
    return dict(
        area=area, 
        tilt_degree=dict(
            horizontal=horizontal_tilt_degree, 
            vertical=vertical_tilt_degree
        ),
        bbox=bbox, 
        vertices=vertices, 
        plate_number=plate_number, 
        brightness=brightness, 
        blurriness=blurriness
    )

In [10]:
# test parse function
parse_file_name_ccpd()

{'area': 25,
 'tilt_degree': {'horizontal': 95, 'vertical': 113},
 'bbox': [154, 383, 386, 473],
 'vertices': [[386, 473], [177, 454], [154, 383], [363, 402]],
 'plate_number': 'AAY339S',
 'brightness': 37,
 'blurriness': 15}

In [11]:
def show_diff(img, info, predicted=None):    
    cv2.rectangle(img, 
                  (int(info["bbox"][0]), int(info["bbox"][1])), 
                  (int(info["bbox"][2]), int(info["bbox"][3])), 
                  (0,120,255), 
                  3)
    if predicted:
        cv2.rectangle(img, 
                  (int(predicted[0]), int(predicted[1])), 
                  (int(predicted[2]), int(predicted[3])), 
                  (255,120,0), 
                  3)
    plt.imshow(img)
    plt.show()

In [12]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [13]:
def compare(path, false_positive=True, false_negative=False, min_iou=0.5, debug=0, res_dir="/var/www/nomeroff-net/datasets/numberplate_china/not_recognizion"):
    if min_iou < 0:
        raise Exception("min_iou mast be grater or equal '0'")
    
    img = cv2.imread(path)
    info = parse_file_name_ccpd(path)
    target_boxes = detector.detect_bbox(copy.deepcopy(img))
    max_iou = 0
    max_bbox = None
    for bbox in target_boxes:
        iou = bb_intersection_over_union(bbox, info["bbox"])
        if false_negative and (not iou or iou < min_iou):
            if debug:
                print("[INFO] path", path)
                print("[INFO] parsed info", info)
                print("[INFO] false_negative iou", max_iou)
                show_diff(img, info, max_bbox)
        if max_iou < iou:
            max_iou = iou
            max_bbox = bbox
    if false_positive and (not max_iou or max_iou < min_iou):
        if debug:
            print("[INFO] path", path)
            print("[INFO] parsed info", info)
            print("[INFO] false_positive iou", max_iou)
            show_diff(img, info, max_bbox)
        copyfile(path, os.path.join(res_dir, os.path.basename(path)))
        return 0
    return 1

In [14]:
%matplotlib inline
#plt.rcParams["figure.figsize"] = (20,10)

In [15]:
GLOB_CCPD_IMAGES = "/var/www/nomeroff-net/datasets/numberplate_china/CCPD2020/ccpd_green/test/*"

good_count = 0
bad_count = 0
for path in tqdm.tqdm(glob.glob(GLOB_CCPD_IMAGES)):
    is_good = compare(path)
    if is_good:
        good_count += 1
    else:
        bad_count += 1
print("[INFO] accuracy:", good_count/(good_count+bad_count))

100%|██████████| 1001/1001 [00:11<00:00, 86.41it/s]

[INFO] accuracy: 0.998001998001998


In [ ]:
suffixes = [
    #"ccpd_base", 
    "ccpd_challenge", 
    "ccpd_db", 
    "ccpd_fn", 
    "ccpd_rotate", 
    "ccpd_tilt", 
    "ccpd_weather",
    "ccpd_blur", 
]
glob_images_format = "/var/www/nomeroff-net/datasets/numberplate_china/CCPD2019/{}/*"

for suffix in suffixes:
    print("[INFO] Start process", suffix)
    glob_images = glob_images_format.format(suffix)
    good_count = 0
    bad_count = 0
    for path in tqdm.tqdm(glob.glob(glob_images)):
        is_good = compare(path, debug=0)
        if is_good:
            good_count += 1
        else:
            bad_count += 1
    print("[INFO] accuracy:", good_count/(good_count+bad_count))

  0%|          | 8/50003 [00:00<10:50, 76.83it/s]

[INFO] Start process ccpd_challenge


  0%|          | 9/10132 [00:00<01:53, 89.25it/s]

[INFO] accuracy: 0.9218646881187129
[INFO] Start process ccpd_db


  0%|          | 9/20967 [00:00<03:55, 88.92it/s]

...

[INFO] accuracy: 0.9439369870267408
[INFO] Start process ccpd_weather


  0%|          | 10/20611 [00:00<03:37, 94.69it/s]

[INFO] accuracy: 0.9901990199019902
[INFO] Start process ccpd_blur


100%|██████████| 20611/20611 [03:38<00:00, 94.17it/s]

[INFO] accuracy: 0.8050070350783562


In [148]:
print("ccpd_green", 0.8997112771739131, 11776*(1-0.8997112771739131))
print("ccpd_rotate", 0.9150502337610663, 10053*(1-0.9150502337610663))
print("ccpd_tilt", 0.9163026211278793, 30216*(1-0.9163026211278793))
print("ccpd_weather", 0.9585958595859586, 9999*(1-0.9585958595859586))
print("ccpd_base", 0.9922348446968939, 199996*(1-0.9922348446968939))
print("ccpd_blur", 0.6204939110183882, 20611*(1-0.6204939110183882))
print("ccpd_challenge", 0.8537887726736396, 50003*(1-0.8537887726736396))
print("ccpd_db", 0.7181208053691275, 10132*(1-0.7181208053691275))
print("ccpd_fn", 0.8068870129250727, 20967*(1-0.8068870129250727))

ccpd_green 0.8997112771739131 1180.9999999999995
ccpd_rotate 0.9150502337610663 854.0000000000003
ccpd_tilt 0.9163026211278793 2528.999999999999
ccpd_weather 0.9585958595859586 413.9999999999999
ccpd_base 0.9922348446968939 1553.0000000000005
ccpd_blur 0.6204939110183882 7822.0
ccpd_challenge 0.8537887726736396 7310.999999999998
ccpd_db 0.7181208053691275 2856.0000000000005
ccpd_fn 0.8068870129250727 4049.000000000001


In [147]:
suffixes = [
            "ccpd_base", "ccpd_blur", "ccpd_challenge", "ccpd_db", "ccpd_fn", 
            "ccpd_rotate", "ccpd_tilt", "ccpd_weather"
]
glob_images_format = "/var/www/nomeroff-net/datasets/numberplate_china/CCPD2019/{}/*"

for suffix in suffixes:
    print("[INFO] Start process", suffix)
    glob_images = glob_images_format.format(suffix)
    print(len(glob.glob(glob_images)))

[INFO] Start process ccpd_base
199996
[INFO] Start process ccpd_blur
20611
[INFO] Start process ccpd_challenge
50003
[INFO] Start process ccpd_db
10132
[INFO] Start process ccpd_fn
20967
[INFO] Start process ccpd_rotate
10053
[INFO] Start process ccpd_tilt
30216
[INFO] Start process ccpd_weather
9999
